<a href="https://colab.research.google.com/github/ahmeda335/Pydantic_Practice_Examples/blob/main/PydanticPracticeExamples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# These are 10 practices on pydantic which contain useful tips and tricks😀

## 🔍🔍 I tried to import public functions first, and then at each single practice, I imported the related functions used there.

In [113]:
from pydantic import BaseModel, Field, ValidationError
from typing import List, Optional, Literal, Annotated
from datetime import datetime

In [114]:
# BaseModel ---> It is the parent model that we inherit it to use the features of pydantic.


# Literal['red', 'green', 'blue', 'yellow'] ---> enforces that the color must be one of the colors mentioned.


#  Annotated in python ---> allows developers to declare the type of a reference and provide additional information related to it.
#  name: Annotated[str, "first letter is capital"]


# ValidationError ---> Error that appears when the instance created isn't similar to the pydantic schema.


# EmailStr ---> It is a datatype to detect the shape of email, but to use it, I must install 'email-validator' as in the next cell.

In [115]:
!pip install email-validator -qU

## 1️⃣ User Registration Schema

In [116]:
from pydantic import EmailStr

In [117]:
#### Goal ####
'''
User Registration Schema
  A user must have:
    - A username (string, at least 5 characters)
    - A password (string, at least 8 characters with one digit)
    - An email (valid email format)
    - An age (integer, at least 18)'''



class Registration(BaseModel):
  username : str = Field(..., min_length=5)
  password : Annotated[str, Field(min_length=8, pattern=r'^[A-Za-z0-9]*[0-9][A-Za-z0-9]*$')]  # This regex pattern to detect the above condition.
  email : EmailStr
  age : int = Field(..., ge=18)  # ge --> greater than or equal.


In [118]:
# Example of valid instance.
try:
  r = Registration(
      username=  "ahmed",
      password= "ahmed30adel",
      email= "ahmedadel@gmail.com",
      age=18
  )
  print(r)
except ValidationError as e:
  print(e)


username='ahmed' password='ahmed30adel' email='ahmedadel@gmail.com' age=18


In [119]:
# Example of invalid instance.
try:
  r = Registration(
      username=  "ahme",
      password= "ahmedadel",
      email= "ahmedadel@gmailcom",
      age=17
  )
  print(r)
except ValidationError as e:
  print(e)

4 validation errors for Registration
username
  String should have at least 5 characters [type=string_too_short, input_value='ahme', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/string_too_short
password
  String should match pattern '^[A-Za-z0-9]*[0-9][A-Za-z0-9]*$' [type=string_pattern_mismatch, input_value='ahmedadel', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/string_pattern_mismatch
email
  value is not a valid email address: The part after the @-sign is not valid. It should have a period. [type=value_error, input_value='ahmedadel@gmailcom', input_type=str]
age
  Input should be greater than or equal to 18 [type=greater_than_equal, input_value=17, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/greater_than_equal


## 2️⃣ Product Inventory Schema

In [120]:
from pydantic import confloat
from annotated_types import Ge

In [121]:
### Goal ###
'''
Product Inventory Schema
  A product must have:
    - A name (string, max length 50)
    - A price (float, must be positive)
    - A stock (integer, must be at least 0)
    - A category (enum: "electronics", "fashion", "books", "home")
    '''


class ProductInventory(BaseModel):
  name : str = Field(max_length=50),
  price : confloat(gt=0.0)
  stock : Annotated[int, Ge(0)]  # This makes sure that the integer is Grater than or equal to zero
  category : Literal["electronics", "fashion", "books", "home"]


In [122]:
# Example of valid instance.
try:
  p = ProductInventory(
      name = 'Printing Machine',
      price = 2250,
      stock = 2,
      category= "electronics"
  )
  print(p)
except ValidationError as e:
  print(e)



name='Printing Machine' price=2250.0 stock=2 category='electronics'


In [123]:
# Example of invalid instance.
try:
  p = ProductInventory(
      name = 'Printing Machine',
      price = -2250,
      stock = -2,
      category= "Mechanic"
  )
  print(p)
except ValidationError as e:
  print(e)

3 validation errors for ProductInventory
price
  Input should be greater than 0 [type=greater_than, input_value=-2250, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/greater_than
stock
  Input should be greater than or equal to 0 [type=greater_than_equal, input_value=-2, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/greater_than_equal
category
  Input should be 'electronics', 'fashion', 'books' or 'home' [type=literal_error, input_value='Mechanic', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/literal_error


## 3️⃣ Blog Post Schema

In [124]:
### Goal ###
'''
Blog Post Schema
  A blog post must have:
    - A title (string, 10-100 characters)
    - A content (string, at least 100 characters)
    - A tags (list of up to 5 strings, max 20 characters each)
    - A published_at (datetime, default to now)
    - An author (nested model containing name and email)
'''

class author(BaseModel):
  name : str
  email : EmailStr

class BlogPost(BaseModel):
  title : Annotated[str, Field(min_length=10, max_length=100)]   # or --> 'constr(min_length=10, max_length=20)'
  content : Annotated[str, Field(min_length=100)]
  tags : Annotated[List[Annotated[str, Field(max_length=20)]], Field(max_items=5)]
  published_at : datetime = Field(default='now'),
  author : author

In [125]:
# Example of valid instance.
try:
  b = BlogPost(
      title = "The Future of AI in Everyday Life",
      content= "Artificial Intelligence is rapidly changing our world. "
        "From smart assistants to self-driving cars, AI is reshaping industries. "
        "In this article, we explore the potential and challenges of AI adoption.",
      tags= ["AI", "technology", "future"],
      published_at="2025-03-04T12:30:00",
      author={
        "name": "John Doe",
        "email": "john.doe@example.com"
    }
  )
  print(b)
except ValidationError as e:
  print(e)

title='The Future of AI in Everyday Life' content='Artificial Intelligence is rapidly changing our world. From smart assistants to self-driving cars, AI is reshaping industries. In this article, we explore the potential and challenges of AI adoption.' tags=['AI', 'technology', 'future'] published_at=datetime.datetime(2025, 3, 4, 12, 30) author=author(name='John Doe', email='john.doe@example.com')


In [126]:
# Example of invalid instance.
try:
  b = BlogPost(
      title = "AI",
      content= "AI is great!",
      tags= ["AI", "ML", "Deep Learning", "NLP", "Data Science", "Big Data"],
      published_at="2025-03-04T12:30:00",
      author={
        "name": "John Doe",
        "email": "john.doeexample.com"
    }
  )
  print(b)
except ValidationError as e:
  print(e)

4 validation errors for BlogPost
title
  String should have at least 10 characters [type=string_too_short, input_value='AI', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/string_too_short
content
  String should have at least 100 characters [type=string_too_short, input_value='AI is great!', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/string_too_short
tags
  List should have at most 5 items after validation, not 6 [type=too_long, input_value=['AI', 'ML', 'Deep Learni...ta Science', 'Big Data'], input_type=list]
    For further information visit https://errors.pydantic.dev/2.10/v/too_long
author.email
  value is not a valid email address: An email address must have an @-sign. [type=value_error, input_value='john.doeexample.com', input_type=str]


## 4️⃣ API Request Validation

In [127]:
import uuid
from pydantic import PastDatetime
from typing import Dict, Any

In [128]:
### Goal ###
'''
API Request Validation
  A request must have:
    - A token (string, exactly 32 characters)
    - A user_id (UUID)
    - A timestamp (datetime, must be in the past)
    - A payload (dictionary with string keys and any JSON-serializable values)
'''

class RequestValidation(BaseModel):
  token: Annotated[str, Field(min_length=32, max_length=32)]   # or ---> 'constr(min_length=32, max_length=32)'
  user_id: uuid.UUID
  timestamp: PastDatetime
  payload: Dict[str, Any] # Any JSON-serializable value.

In [129]:
# Example of valid instance.
try:
  r = RequestValidation(
      token = "a1b2c3d4e5f6g7h8i9j0k1l2m3n4o5p6",
      user_id="550e8400-e29b-41d4-a716-446655440000",
      timestamp= "2024-03-04T12:30:00",
      payload= {"username": "johndoe",
        "score": 95,
        "verified": True,
        "preferences": {"theme": "dark", "notifications": False},
        "items": ["apple", "banana"],
        "discount": None
      }
  )
  print(r)
except ValidationError as e:
  print(e)

token='a1b2c3d4e5f6g7h8i9j0k1l2m3n4o5p6' user_id=UUID('550e8400-e29b-41d4-a716-446655440000') timestamp=datetime.datetime(2024, 3, 4, 12, 30) payload={'username': 'johndoe', 'score': 95, 'verified': True, 'preferences': {'theme': 'dark', 'notifications': False}, 'items': ['apple', 'banana'], 'discount': None}


In [130]:
# Example of invalid instance.
try:
  r = RequestValidation(
      token = "shorttoken123",
      user_id="12345",
      timestamp= "2030-01-01T12:00:00",
      payload= {"username": "johndoe",
        42: 95,
        "verified": True,
        "preferences": {"theme": "dark", "notifications": False},
        "items": ["apple", "banana"],
        "discount": None
      }
  )
  print(r)
except ValidationError as e:
  print(e)

4 validation errors for RequestValidation
token
  String should have at least 32 characters [type=string_too_short, input_value='shorttoken123', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/string_too_short
user_id
  Input should be a valid UUID, invalid length: expected length 32 for simple format, found 5 [type=uuid_parsing, input_value='12345', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/uuid_parsing
timestamp
  Input should be in the past [type=datetime_past, input_value='2030-01-01T12:00:00', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/datetime_past
payload.42.[key]
  Input should be a valid string [type=string_type, input_value=42, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type


##5️⃣ Employee Management System

In [131]:
from pydantic import PositiveInt, PastDatetime

In [132]:
### Goal ###
'''
Employee Management System
  An employee must have:
    - A name (string, non-empty)
    - An employee_id (positive integer)
    - A salary (float, at least 30000)
    - A role (enum: "manager", "developer", "designer")
    - A joining_date (date, must be before today)
'''

class EmployeeManagementSystem(BaseModel):
  name: str = Field(..., min_length=1)
  employee_id: PositiveInt
  salary: Annotated[float, Field(ge=30000)]
  role: Literal["manager", "developer", "designer"]
  joining_date: PastDatetime

In [133]:
# Example of valid instance.
try:
  e = EmployeeManagementSystem(
      name = 'Ahmed',
      employee_id = 123,
      salary= 120_000,
      role='manager',
      joining_date = "2024-03-30T12:00:00",
  )
  print(e)
except ValidationError as e:
  print(e)

name='Ahmed' employee_id=123 salary=120000.0 role='manager' joining_date=datetime.datetime(2024, 3, 30, 12, 0)


In [134]:
# Example of invalid instance.
try:
  e = EmployeeManagementSystem(
      name = '',
      employee_id = -123,
      salary= 20000,
      role='architecture',
      joining_date = "2024-03-3012:00:00",
  )
  print(e)
except ValidationError as e:
  print(e)

5 validation errors for EmployeeManagementSystem
name
  String should have at least 1 character [type=string_too_short, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/string_too_short
employee_id
  Input should be greater than 0 [type=greater_than, input_value=-123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/greater_than
salary
  Input should be greater than or equal to 30000 [type=greater_than_equal, input_value=20000, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/greater_than_equal
role
  Input should be 'manager', 'developer' or 'designer' [type=literal_error, input_value='architecture', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/literal_error
joining_date
  Input should be a valid datetime or date, unexpected extra characters at the end of the input [type=datetime_from_date_parsing, input_value='2024-03-3012

## 6️⃣ Order Processing System

In [135]:
import uuid
from pydantic import PositiveInt

In [136]:
### Goal ###
'''
Order Processing System
  An order must have:
    - An order_id (UUID)
    - A customer_name (string, min 3 characters)
    - A items (list of dictionaries with item_name and quantity, where quantity is a positive integer)
    - A total_price (float, calculated from items)
    - A status (enum: "pending", "shipped", "delivered", "cancelled")
'''

class Item(BaseModel):
  item_name: str = Field(..., min_length=1)
  quantity: PositiveInt    # Or 'int = Field(..., gt=0)'
  price: float = Field(..., gt=0)

class OrderProcessingSystem(BaseModel):
  order_id: uuid.UUID
  customer_name: str = Field(..., min_length=3)
  items: List[Item]
  status: Literal["pending", "shipped", "delivered", "cancelled"]

  @property
  def total_price(self) -> float:
    return sum(item.price * item.quantity for item in self.items)

In [137]:
# Example of valid instance.
try:
  o = OrderProcessingSystem(
      order_id = "550e8400-e29b-41d4-a716-446655440000",
      customer_name= "Ahmed",
      items = [
          {
              "item_name" : 'laptop',
              "quantity" : 2,
              "price" : 1000,
          },
          {
              "item_name" : 'mouse',
              "quantity" : 10,
              "price" : 80
          }
      ],
      status = 'delivered'
  )
  print(o)
  print("Total Salary: ", o.total_price)
except ValidationError as e:
  print(e)

order_id=UUID('550e8400-e29b-41d4-a716-446655440000') customer_name='Ahmed' items=[Item(item_name='laptop', quantity=2, price=1000.0), Item(item_name='mouse', quantity=10, price=80.0)] status='delivered'
Total Salary:  2800.0


In [138]:
# Example of invalid instance.
try:
  o = OrderProcessingSystem(
      order_id = "550e8400-e29b-41d4-a716",
      customer_name= "Ah",
      items = [
          {
              "item_name" : 'laptop',
              "quantity" : 2,
              "price" : -1000,
          },
          {
              "item_name" : 'mouse',
              "quantity" : 10,
              "price" : 80
          }
      ],
      status = 'delive'
  )
  print(o)
  print("Total Salary: ", o.total_price)
except ValidationError as e:
  print(e)

4 validation errors for OrderProcessingSystem
order_id
  Input should be a valid UUID, invalid group count: expected 5, found 4 [type=uuid_parsing, input_value='550e8400-e29b-41d4-a716', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/uuid_parsing
customer_name
  String should have at least 3 characters [type=string_too_short, input_value='Ah', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/string_too_short
items.0.price
  Input should be greater than 0 [type=greater_than, input_value=-1000, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/greater_than
status
  Input should be 'pending', 'shipped', 'delivered' or 'cancelled' [type=literal_error, input_value='delive', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/literal_error


## 7️⃣ IoT Sensor Data Schema

In [139]:
import uuid

In [140]:
### Goal ###
'''
IoT Sensor Data Schema
  A sensor must have:
    - A sensor_id (UUID)
    - A temperature (float, between -50 and 150)
    - A humidity (float, between 0 and 100)
    - A status (boolean, active or inactive)
    - A timestamp (datetime, default to now)
'''


class IoTSensorDataSchema(BaseModel):
  sensor_id: uuid.UUID
  temperature: float = Field(..., ge=-50, le=150)
  humidity: float = Field(..., ge=0, le=100)
  status: Literal['active', 'inactive']
  timestamp: datetime = Field(default='now')

In [141]:
# Example of valid instance.
try:
  i = IoTSensorDataSchema(
      sensor_id= "550e8400-e29b-41d4-a716-446655440000",
      temperature= 32,
      humidity= 23,
      status= 'active',
  )
  print(i)
except ValidationError as e:
  print(e)

sensor_id=UUID('550e8400-e29b-41d4-a716-446655440000') temperature=32.0 humidity=23.0 status='active' timestamp='now'


In [142]:
# Example of invalid instance.
try:
  i = IoTSensorDataSchema(
      sensor_id= "e29b-41d4-a716-446655440000",
      temperature= -72,
      humidity= -23,
      status= 'aactive',
  )
  print(i)
except ValidationError as e:
  print(e)

4 validation errors for IoTSensorDataSchema
sensor_id
  Input should be a valid UUID, invalid group count: expected 5, found 4 [type=uuid_parsing, input_value='e29b-41d4-a716-446655440000', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/uuid_parsing
temperature
  Input should be greater than or equal to -50 [type=greater_than_equal, input_value=-72, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/greater_than_equal
humidity
  Input should be greater than or equal to 0 [type=greater_than_equal, input_value=-23, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/greater_than_equal
status
  Input should be 'active' or 'inactive' [type=literal_error, input_value='aactive', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/literal_error


## 8️⃣ Student Records Schema

In [143]:
from pydantic import EmailStr

In [144]:
### Goal ###
'''
Student Records Schema
  A student must have:
    - A full_name (string, min 3 characters)
    - An email (valid email)
    - A gpa (float, between 0 and 4.0)
    - A courses (list of dictionaries, each with course_name and credits where credits is 1-6)
'''

class Course(BaseModel):
  course_name: str = Field(..., min_length=1)
  credits: int = Field(..., ge=1, le=6)

class StudentRecordsSchema(BaseModel):
  full_name: str = Field(..., min_length=3)
  email: EmailStr
  gpa: float = Field(..., ge=0.0, le=4.0)
  courses: List[Course]

In [145]:
# Example of valid instance.
try:
  s = StudentRecordsSchema(
      full_name= "Ahmed Adel",
      email= 'ahmed30@gmail.com',
      gpa= 3.9,
      courses= [
          {
              "course_name": "Database",
              "credits": 4
          },
          {
              "course_name": "OS",
              "credits": 5
          }
      ]
  )
  print(s)
except ValidationError as e:
  print(e)

full_name='Ahmed Adel' email='ahmed30@gmail.com' gpa=3.9 courses=[Course(course_name='Database', credits=4), Course(course_name='OS', credits=5)]


In [146]:
# Example of invalid instance.
try:
  s = StudentRecordsSchema(
      full_name= "Ah",
      email= 'ahmed30gmail.com',
      gpa= -3.9,
      courses= [
          {
              "course_name": "Database",
              "credits": 7
          },
          {
              "course_name": "OS",
              "credits": 5
          }
      ]
  )
  print(s)
except ValidationError as e:
  print(e)

4 validation errors for StudentRecordsSchema
full_name
  String should have at least 3 characters [type=string_too_short, input_value='Ah', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/string_too_short
email
  value is not a valid email address: An email address must have an @-sign. [type=value_error, input_value='ahmed30gmail.com', input_type=str]
gpa
  Input should be greater than or equal to 0 [type=greater_than_equal, input_value=-3.9, input_type=float]
    For further information visit https://errors.pydantic.dev/2.10/v/greater_than_equal
courses.0.credits
  Input should be less than or equal to 6 [type=less_than_equal, input_value=7, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/less_than_equal


## 9️⃣ Car Rental System

In [147]:
import uuid
from pydantic import validator

In [148]:
### Goal ###
'''
Car Rental System
  A rental record must have:
    - A car_id (UUID)
    - A customer_name (string, min 3 characters)
    - A rental_start (datetime)
    - A rental_end (datetime, must be after rental_start)
    - A price_per_day (float, positive)
    - A total_price (float, calculated based on rental days)
'''


class CarRentalSystem(BaseModel):
  car_id: uuid.UUID
  customer_name: str = Field(..., min_length=3)
  rental_days: int = Field(..., gt=0)
  rental_start: datetime
  rental_end: datetime
  price_per_day: float = Field(..., gt=0)

  @property
  def total_price(self) -> float:
    return self.price_per_day * self.rental_days

  @validator("rental_end")
  def validate_rental_end_bigger_than_rental_start(cls, rental_end, values):
    rental_start = values.get("rental_start")
    if rental_start and rental_end <= rental_start:
      raise ValueError("rental_end must be after rental_start")
    return rental_end

<ipython-input-148-45ea5f4c1803>:26: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("rental_end")


In [149]:
# Example of valid instance.
try:
  c = CarRentalSystem(
      car_id = "550e8400-e29b-41d4-a716-446655440000",
      customer_name = "Ahmed",
      rental_days = 3,
      rental_start = "2025-03-01T12:00:00",
      rental_end = "2025-03-04T12:00:00",
      price_per_day = 100
  )
  print(c)
except ValidationError as e:
  print(e)

car_id=UUID('550e8400-e29b-41d4-a716-446655440000') customer_name='Ahmed' rental_days=3 rental_start=datetime.datetime(2025, 3, 1, 12, 0) rental_end=datetime.datetime(2025, 3, 4, 12, 0) price_per_day=100.0


In [150]:
# Example of invalid instance.
try:
  c = CarRentalSystem(
      car_id = "550e8400-e29b-41d4-a716-446655440000",
      customer_name = "Ahmed",
      rental_days = 3,
      rental_start = "2025-03-01T12:00:00",
      rental_end = "2024-03-04T12:00:00",   # I changed only this.
      price_per_day = 100
  )
  print(c)
except ValidationError as e:
  print(e)

1 validation error for CarRentalSystem
rental_end
  Value error, rental_end must be after rental_start [type=value_error, input_value='2024-03-04T12:00:00', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error


## 🔟 Cryptocurrency Transaction Validation

In [151]:
import uuid
from pydantic import PastDatetime

In [152]:
### Goal ###
'''
Cryptocurrency Transaction Validation
  A transaction must have:
    - A transaction_id (UUID)
    - A sender_wallet (string, exactly 34 characters)
    - A receiver_wallet (string, exactly 34 characters)
    - An amount (float, at least 0.0001)
    - A timestamp (datetime, must be in the past)

'''

class Cryptocurrency(BaseModel):
  transaction_id: uuid.UUID
  sender_wallet: str = Field(..., min_length= 34, max_length= 34)
  receiver_wallet: str = Field(..., min_length= 34, max_length= 34)
  amount: float = Field(..., ge=0.0001)
  timestamp: PastDatetime

In [153]:
# Example of valid instance.
try:
  c = Cryptocurrency(
      transaction_id="550e8400-e29b-41d4-a716-446655440000",
      sender_wallet= "dkem39ekc9ekls9ekvjeljokdjvldjeljf",
      receiver_wallet= "dkem39ekc9ekls9ekvjeljokdjvldjeknk",
      amount= 3.5,
      timestamp="2024-03-04T12:00:00"
  )
  print(c)
except ValidationError as e:
  print(e)

transaction_id=UUID('550e8400-e29b-41d4-a716-446655440000') sender_wallet='dkem39ekc9ekls9ekvjeljokdjvldjeljf' receiver_wallet='dkem39ekc9ekls9ekvjeljokdjvldjeknk' amount=3.5 timestamp=datetime.datetime(2024, 3, 4, 12, 0)


In [154]:
# Example of invalid instance.
try:
  c = Cryptocurrency(
      transaction_id="550e840-41d4-a716-446655440000",
      sender_wallet= "dkem39ekckls9ekvjeljokdjvldjeljf",
      receiver_wallet= "dkem3ls9ekvjeljokdjvldjeknk",
      amount= 0,
      timestamp="2027-03-04T12:00:00"
  )
  print(c)
except ValidationError as e:
  print(e)

5 validation errors for Cryptocurrency
transaction_id
  Input should be a valid UUID, invalid group count: expected 5, found 4 [type=uuid_parsing, input_value='550e840-41d4-a716-446655440000', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/uuid_parsing
sender_wallet
  String should have at least 34 characters [type=string_too_short, input_value='dkem39ekckls9ekvjeljokdjvldjeljf', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/string_too_short
receiver_wallet
  String should have at least 34 characters [type=string_too_short, input_value='dkem3ls9ekvjeljokdjvldjeknk', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/string_too_short
amount
  Input should be greater than or equal to 0.0001 [type=greater_than_equal, input_value=0, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/greater_than_equal
timestamp
  Input should be in the past [type=